Author: Brian Erichsen Fagundes
MSD CS 6017 - Summer - 2024
Homework 3: Scraping and Regression

Part 1 Data Acquisiton

In [27]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import re


# Create arrays of data to be scraped
ranks = []
title_lengths = []
age_in_hours = []
points = []
comments_numbers = []

for i in range(5):
    url="http://news.ycombinator.com/?p=" + str(i+1)
# Access website contents
    with urllib.request.urlopen (url) as response:
        html=response.read()
        html=html.decode("utf-8")

# Saves html content into file
    with open ("hackernews" + str(i+1) + ".html", "w") as new_file:
        new_file.write(html)

# Parses html content into a soup
    soup = BeautifulSoup(html, 'html.parser')
    
# Scrapes rank data
    for post in soup.find_all(class_="rank"):
        rank = str(post.text)
        rank = rank.replace('.', '')
        ranks.append(int(rank))

# Loads length of title data
    for title in soup.find_all(class_="titleline"):
        title_lengths.append(len(title.text))

# Loads age data
    for age in soup.find_all(class_="age"):
        age_str =  str(age.text)
        age_str =  age_str.removesuffix(" hours ago")
        age_str = age_str.removesuffix(" hour ago")

        if " day ago" in age_str:
            age_in_hours.append(24)
        elif " days ago" in age_str:
            string = age_str.replace(" days ago", "")
            num = int(string)
            age_in_hours.append(num*24)
        elif " minute ago" in age_str or " minutes ago" in age_str:
            age_in_hours.append(0)
        else:
            age_in_hours.append(int(age_str))

    for subtext in soup.find_all(class_="subtext"):
        point = 0
        comments = 0
        for score in subtext.find_all(class_="score"):
            point = int(re.search(r'\d+', str(score.text)).group())

        for tag in subtext.find_all("a"):
            if tag.text.endswith("comments"):
                a_= str(tag.text)
                comments = int(re.search(r'\d+', a_).group())

        points.append(int(point))
        comments_numbers.append(int(comments))

data_frame = pd.DataFrame({
    "Rank" : ranks, "Title Length" : title_lengths, "Age (hours)" : age_in_hours,
    "Points" : points, "Comments" : comments_numbers
})
print(data_frame.head())


   Rank  Title Length  Age (hours)  Points  Comments
0     1            57            3     102        16
1     2            72            7     244        96
2     3            55            4      45         2
3     4            79           16     384       223
4     5            53            8     109        20


In [28]:
# save data into file so we don't have to load data each time
data_frame.to_csv("hacker_news_stories.csv", index=False)

Part 2 - Regression